# 1. Multiple dispatch
Multiple dispatch is the core programming paradigm of Julia. 

## 1.1. What is Multiple Dispatch?

Some handy definitions
* **function**: the name of the "function / process" we are referring to.
* **method**: what actually happens when we call the function.

*Dispatch* means that when a function call occurs, the language decides somehow which of the function *methods* have to be used. 

#### No dispatch
In no dispatch, as in e.g. C, there is nothing to be decided. The method and the function is the same thing. 

#### Single dispatch
In single dispatch, as in most object-oriented languages (like Python), it is possible for the same function name to have different methods:
```
array.set_size(args...)
axis.set_size(args...)
```
where `array` could be an instance of something from `numpy` while `axis` could come from `matplotlib`. Here the language dispatches the function `set_size`, depending on the first argument, which is `array` or `axis`. It is important to note that in most single dispatch languages, the **method is a property of the type**. Notice also that the rest of the arguments `(args...)` do not contribute in any way into dispatch.

#### Multiple dispatch
Here dispatch occurs based on the type of **every single function argument**, as in
```
set_size(a::Array, args...) = ...
set_size(a::Axis, args...) = ...
set_size(s, a::Array, args...) = ...
set_size(a::Array, b::Vector) = ...
set_size(a::Array, x::Real, y::Real, z::Real) = ...
```
etc. Multiple dispatch follows easy-to-understand rules based on the Julia type system hierarchy. Two important points:

1. **methods do not belong to the objects!**
2. **new methods can be defined *after* the types have been defined!**

*operator overloading is not multiple dispatch, see below* 

#### Video talk
A large part of this notebook is based on a talk by [Stefan Karpinski at JuliaCon2019](https://www.youtube.com/watch?v=kc9HwsxE1OY), which is a more in depth look at multiple dispatch and why it is really awesome. If you want to know more about multiple dispatch, I highly recommend.




## 1.2. Expressive power of multiple dispatch

| dispatch |     syntax     | expressive order | expressive power |   |
|:--------:|:--------------:|:----------------:|:----------------:|:-:|
|   none   | f(x1, x2, ...) |       O(1)       |     constant     |   |
|  single  |  x1.f(x2, ...) |       O(X1)      |      linear      |   |
| multiple | f(x1, x2, ...) |   O(X1⋅X2⋅...)   |    exponential   |   |

## 1.3. A simple example

Multiple dispatch builds upon Julia's type system hierarchy. 

#### Abstract and concrete types
Abstract types can be defined, and concrete types can be subtypes of the abstract types. Multiple dispatch can be defined on both abstract and concrete types, but only concrete types can be instantiated (i.e. actually exist).

#### Method dispatch
Upon calling a function, Julia will try to find the method that is most specialized across all arguments. This means that if a method is defined for both the abstract type combination, as well as the concrete type combination, Julia will always call the more specialized one.

#### Defining some types

In [1]:
abstract type Animal end # this is an abstract type. a supertype of the below

struct Dog <: Animal   # this is a concrete type. a subtype of the above
    name::String
end

struct Cat <: Animal
    name::String
end

Now let's instantiate four animals

In [2]:
fido = Dog("Fido")
rex = Dog("Rex")
whiskers = Cat("Whiskers")
spotty = Cat("Spotty")

Cat("Spotty")

And finally, let's define some functions that take advantage of these `Animal` types, as well as multiple dispatch:

In [3]:
function encounter(a::Animal, b::Animal)
    verb = meets(a, b)
    println("$(a.name) meets $(b.name) and $verb")
end

meets(a::Animal, b::Animal) = "passes by"

meets (generic function with 1 method)

Both of the above functions are defined on the abstract type level, and thus are not specific.

In [4]:
encounter(fido, rex)

Fido meets Rex and passes by


We now define more specific methods like so:

In [5]:
meets(a::Dog, b::Dog) = "sniffs"
meets(a::Dog, b::Cat) = "chases"
meets(a::Cat, b::Dog) = "hisses"
meets(a::Cat, b::Cat) = "slinks"

meets (generic function with 5 methods)

In [6]:
encounter(fido, rex)
encounter(fido, whiskers)
encounter(whiskers, spotty)
encounter(spotty, rex)

Fido meets Rex and sniffs
Fido meets Whiskers and chases
Whiskers meets Spotty and slinks
Spotty meets Rex and hisses


## 1.4. Simple extention of multiple dispatch
What if we get a third animal? Like a rabbit? It is easy to extend the system

In [7]:
struct Rabbit <: Animal
    name::String
end

meets(a::Dog, b::Rabbit) = "wiggles its tail"
meets(a::Rabbit, b::Cat) = "hides"

hops = Rabbit("Hops")

Rabbit("Hops")

In [8]:
encounter(hops, whiskers)
encounter(rex, hops)
encounter(whiskers, hops)

Hops meets Whiskers and hides
Rex meets Hops and wiggles its tail
Whiskers meets Hops and passes by


I won't be going into Modules and namespaces here. But it is important to note that it would be possible for us to extend `Pet` as well as `meets` *even if* they were coming from a different Module.

## 1.5. Inspecting dispatch

To see how many methods a function has to it, and from which module they come, use `methods`:

In [9]:
methods(meets)

# 7 methods for generic function "meets":
[1] meets(a::Rabbit, b::Cat) in Main at In[7]:6
[2] meets(a::Dog, b::Rabbit) in Main at In[7]:5
[3] meets(a::Cat, b::Cat) in Main at In[5]:4
[4] meets(a::Cat, b::Dog) in Main at In[5]:3
[5] meets(a::Dog, b::Cat) in Main at In[5]:2
[6] meets(a::Dog, b::Dog) in Main at In[5]:1
[7] meets(a::Animal, b::Animal) in Main at In[3]:6

To see which method is called on a function call signature, use `@which`:

In [10]:
@which meets(whiskers, rex)

meets(a::Cat, b::Dog) in Main at In[5]:3

In Julia, all "generic" functions are equivalent. The function `+` is in no way superior, or with more power, than the function `meets` we wrote up.

In [11]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:96
[12] +(::Missing) in Base at missing.jl:83
[13] +(::Missing, ::Number) in Base at missing.jl:97
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:406
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:434
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:433
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:432
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:403
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt8}) in Base.GMP at gmp.jl:440
[21] +(x::BigInt, c::Union{Int16, Int32, Int8}) in Base.GMP at gmp.jl:446
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:556
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:549
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:543
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:402
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:371
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt8}) in Base.MPFR at mpfr.jl:378
[28] +(x::BigFloat, c::Union{Int16, Int32, Int8}) in Base.MPFR at mpfr.jl:386
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:394
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\stdlib\v1.1\Dates\src\arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at C:\cygwin\home\Administrator\buildbot\worker\package_win64\build\usr\share\julia\s

In [12]:
import Base: + # this allows us to extend the function + 
# we import `+` because it comes from another module (Base)

+(a::Animal, b::Animal) = println("$(a.name) and $(b.name) are animals")
+(a::Cat, b::Cat) = println("$(a.name) and $(b.name) cutie-cats, much cuter than dogs!")

+ (generic function with 165 methods)

In [13]:
rex + whiskers

Rex and Whiskers are animals


In [14]:
spotty + whiskers

Spotty and Whiskers cutie-cats, much cuter than dogs!


## 1.6. Obvious benefits of multiple dispatch

- Expressive power to the user
- Generic algorithms can be applied to wildly different types, by extending key functions
- Common types can be shared by very different packages
- Ability to extend packages of other people, without accessing their code (and vice versa)
- The process and the data structure are separate entities, which is more in line with scientific thought

In the following sections we will go into more detail in some of these points.

## 1.7. Parametric types

Julia types can be parameterized based on other types. This is very useful for reducing code replication as well as leveraging multiple dispatch more. Type-parameterization means that when defining a `struct`, the fields of the struct could be of aribtrary parameter. Type-parameterization uses the curly brakets syntax. For example:

In [15]:
struct Alpha{T}
    a::T
    b::T
end

Here the type `Alpha` can be instantiated using the low level constructor `Alpha{Type}(val1, val2)`, which will convert automatically `val1, val2` to `Type`. For example

In [16]:
Alpha{Float64}(1, π)

Alpha{Float64}(1.0, 3.141592653589793)

The high level constructor `Alpha(val1, val2)` also exist, and will try to deduce what `Type` should be based on the arguments:

In [17]:
Alpha(0.5, 0.6)

Alpha{Float64}(0.5, 0.6)

In [18]:
Alpha("test", 0.5)

MethodError: MethodError: no method matching Alpha(::String, ::Float64)
Closest candidates are:
  Alpha(::T, !Matched::T) where T at In[15]:2

Since a string and a float are different types, one cannot instantiate `A` like this.

Here is a second example with two type-parameters:

In [19]:
struct B{T, F}
    a::T
    b::F
end

In [20]:
B("string", 0.5)

B{String,Float64}("string", 0.5)

And a final example:

In [21]:
struct C{T <: Real, F <: Union{String, Real}}
    a::T
    b::F
    c::Int
end

Here, to instantiate `C`, `a` must be a subtype of reals, `b` could be a string or a real, while `c` has to be an integer.

**In general, all fields of a `struct` should be strictly typed.**

Multiple dispatch is of course respecting parametric types. For example if we define:

In [32]:
foo(a::Alpha) = "generic function"
foo(a::Alpha{Float64}) = "specific to A with floats"

foo (generic function with 2 methods)

then

In [35]:
foo(Alpha(1,2))

"generic function"

In [36]:
foo(Alpha(0.5, 0.6))

"specific to A with floats"

For more info on the type system and multiple dispatch please read the official Julia documentation on [Types](https://pkg.julialang.org/docs/julia/THl1k/1.1.1/manual/types.html) and [Methods](https://pkg.julialang.org/docs/julia/THl1k/1.1.1/manual/methods.html).

# 2. Generic algorithms

How does one go about writing a generic algorithm through multiple dispatch? Well, it's simple really. Just write the code that does what you want in a simple and general manner. Julia will take care of the rest.

## 2.1. Some linear algebra algorithm
Imagine the following simple algorithm, that performs some linear algebra computations involving inner products. `A` is supposed to be a matrix, while `vs` is supposed to be a vector of vectors.

In [25]:
using LinearAlgebra

function inner_sum(A, vs)
    t = zero(eltype(A))
    for v in vs
        t += inner(v, A, v)
    end
    return t
end

inner(v, A, w) = dot(v, A*w)

inner (generic function with 1 method)

In [26]:
A = rand(3,3)
vs = [rand(3) for i in 1:5]
inner_sum(A, vs)

6.397531322691732

* The above code is **generic**: any `A, vs` that support the correct operations (here `*` and `dot` only), will work fine.

* Because Julia *compiles* all code based on the exact input types, `inner_sum` is also **arbitrarily efficient**. Given standard Julia matrices and vectors it uses BLAS. Given statically-sized arrays it uses super-efficient unrolled operations.

## 2.2. The "OneHot" vector

In various fields that use linear algebra, like machine learning, a special vector datastracture is used often, called a "OneHot" vector. This vector has a 1 in a single entry, and 0 in all other entries. Having a dedicated structure for this vector is super efficient, because you only need to store 2 numbers to store it in memory, irrespectively of how large it is!

Let's create this in Julia.

In [27]:
struct OneHotVector <: AbstractVector{Bool}
    len::Int
    ind::Int
end

# Extend size (so that it behaves like an array)
Base.size(v::OneHotVector) = (v.len, )

# Extend indexing:
Base.getindex(v::OneHotVector, i::Integer) = i == v.ind

Does our algorithm already work with such a vector?

In [28]:
vs = [OneHotVector(3, rand(1:3)) for _ in 1:5]

inner_sum(A, vs)

1.7354113043006616

## 2.3. OneHot inner product

It is already great that the above algorithm works with the new `OneHotVector` type. The generic code that one writes with Julia is extensible with **minimal** effort.

*But*, it currently uses the generic fallback `inner(w, A, v) = dot(w, A*v)`. This is inneficient! Let's break this down:

1. `A*v` is a matrix-vector multiplication. It iterates every column of `A` and multiplies them by each entry of `v`.
2. If the vector `v` has 0s everywhere and a 1 somewhere, then this operation in fact means `A[:, v.ind]`. I.e. we select the `v.ind`-th column of the matrix `A`.
3. `dot(w, Av)` is also generic, it indexes into `w` and `Av` and does a pair-wise multiplication.
4. Given the structure of `OneHotVector` we know that `inner(w, A, v)` is in fact just `A[w.ind, v.ind]`!

Increasing the performance of our algorithm is **trivial**. Simply extend `inner` for `OneHotVector`. But first, some benchmarks...

In [29]:
using BenchmarkTools

L = 100
A = rand(L, L)
vs = [OneHotVector(L, rand(1:L)) for _ in 1:5]

# timing before performance extension
@btime inner_sum(A, vs)

  12.799 μs (6 allocations: 4.39 KiB)


3.5859134762581295

In [30]:
inner(w::OneHotVector, A, v::OneHotVector) = A[w.ind, v.ind]

inner (generic function with 2 methods)

In [31]:
# timing after performance extension
@btime inner_sum(A, vs)

  30.294 ns (1 allocation: 16 bytes)


3.5859134762581295

Such a generic algorithm (and its extention) are **impossible in object oriented languages like Python**. For more details please see the talk "The unreasonable effectiveness of multiple dispatch" by Stefan Karpinski on YouTube.

# 3. Code re-use and the Julia ecosystem

There is an astoning amount of code-reuse and cooperation among different packages of the Julia ecosystem. Naively one would think it is because of the ease of writing generic algorithms. This is in part true, but the reason Julia is the only language that truly succeeds in having all packages play nicely with each other is that you can re-use types from other people!

---

## 3.1. An implemented type

Let's take as example the type `RGB`, which represents a color, and is defined in the Julia package `ColorTypes`. It is a simple type that simple bundles a value for red, green and blue, as well as defines some basic operations for the `RGB` type.

## 3.2. Extending this type
Now someone, for some reason, wants to **extend this existing type** `RGB`. For example, one might want to create a color vector space. 

How is this done in Julia?

Simple! Simply extend the functionality of `RGB` by adding new methods that dispatch on `RGB`. For example, you could extend **existing operations**
```julia
module ExtentionsToRGB # creates a new module that 

using RGB

Base.zero(::RGB) = RGB(0,0,0)
Base.rand(::RGB) = RGB(rand(), rand(), rand())

end
```
etc., or **define new operations**
```julia
# dot product in color space: convert to grayscale and then normalize
dotc(x::RGB, y::RGB) = 0.2*x.r*y.r + 0.771*x.g*y.g + 0.029*x.b*y.b
```

All of this "just works (TM)" in Julia, **even if the type `RGB` comes from a completely independent Julia package!**

## 3.3. Is this possible in OOP ?
Seems like the above is something completely trivial to do in Julia. How possible it is in object-oriented programming?

> **It is almost completely impossible**

In OOP the methods of a "class" (which for Julia is a Type), are **defined literally textually within the class**. Therefore, if you wanted to extend this `RGB` in Python you would have two options:

1. Edit the original class.
2. Inherit the original class.

### 3.3.1 Modifying the original class
There are problems here:
* You have to convice the author of the original package that your code is worthy to be included.
* If everyone convinces the author about their method, the original class becomes huge and the source code becomes an unreadable mess.
* The original developer has to maintain everyone else's whacka-doodle methods.
* You can't change your mind about your extentions without potentially breaking everyone that uses the class `RGB`.

### 3.3.2. Inheriting
There are problems here as well:
* If you inherit, you can't "inherit the name"; you have to create a new type `MyRGB` that inherits `RGB`.
* Your extention will **not** actually apply to `RGB`, only to `MyRGB`. Therefore, anyone that creates `RGB` cannot potentially benefit from your code.
* This complexity quickly increases exponentially; someone else makes `YourRGB` object, and then a third person tries to inherit from both `MyRGB` and `YourRGB` and makes a `OurRGB`, etc... Quickly becomes a hellhole!

### 3.3.3. Julia is superior
Julia's multiple dispatch has none of the problems of 3.3.1 and 3.3.2. In addition, everyone that does not load the module `ExtentionsToRGB` is completly unaffected by whatever whacka-doodle shady business is happening there. On the other side, with the simple one liner `using ExtentionsToRGB`, you immediatelly gain access to all the extentions that have been done. **All this with using the same, single Julia type `RGB`**. No `MyRGB` nonsense!

The reasons why this is so trivial in Julia is because:
1. You can define new types to which *existing operations* apply
2. You can define new operations which apply to *existing types*

## 3.4. The Julia ecosystem
The above might seem like a fairy tail. Yet, this is *exactly* how the Julia ecosystem works. Packages re-use the code of other packages and leverage the advantage of generic algorithms.

For example, the module `OrdinaryDiffEq` provides hunderds of solvers for sets of ordinary differential equations. These solvers (by default) work with numbers, vectors, arrays, etc. The module `Measurements` provide numeric types that have error bars; so that your numeric operations propagate the errors as is done in experimental physics.

Without any effort, you can solve a differential equation of numbers with error bars! And your solution will itself be a sequence of numbers with properly calculated errorbars.

* How does this happen? Because `OrdinaryDiffEq` uses internally standard mathematical operations to solve an ODE. `Measurements` by itself naturally defines arithmetics on `+, -, *,...` etc. on numbers with errors.
* `Measurements` has never written any code that solves a differential equation.
* `OrdinaryDiffEq` has never written any code that controls how an error is propagated.

This is how it looks:

![](measurements_diffeq.png)

Figure taken from "The unreasonable effectiveness of multiple dispatch" by Stefan Karpinski, YouTube.

# 4. Exercises

## 4.1. Type hierarchies

Create a function `person_info(p)` that takes in any type of a "person" and prints their name and some extra information. For a normal person, only the name is printed. If the person is a student, it should print their name *and* grade. If it is a group leader, then print their name and their group name. If the input to `person_info` is something other than a "person", it should error.

Solve this exercise without using a single `if` statement; only multiple dispatch. It is to your benefit to define some abstract types.

These kind of problems seem to be "natural" to solve using `if` statements. However using multiple dispatch instead makes the code clearer, more easy to extend and **much** more performant, because of how Julia compiles code.

## 4.2. Indexing of a range object
Create your own `Range` object, which is an efficient iterable container defined by a `(start, step, stop)`. This object can efficiently represent this range with storing only these 3 numbers, instead of all elements that theoretically belong to the range.

1. As a first step, define your `Range` `struct`, preferably by making it a parametric type of 1 type-parameter.
2. Then use the help functionality to learn about the function `getindex`. The Julia syntax `A[1]` is translated to `getindex(A, 1)`.
3. Implement indexing for your Range type, with the index `1` giving `start`. Be sure it errors for incorrect indices.

Hint: make your type parameteric. 

## 4.3. Iteration of a range object

Continuing from the previous exercise, now make your `Range` object iterable. 

1. Read on the `iterate` function and the [Julia documentation on the iteration api](https://pkg.julialang.org/docs/julia/THl1k/1.1.1/base/collections.html#lib-collections-iteration-1). The Julia code block `for a in v; ...; end` is actually using only the `iterate` function.
2. Extend `iterate` for your `Range`, and test that it is indeed iterable by using it in a `for` loop.


* **Bonus point** Did you write generic code? Try to see if your `Range` works with `Unitful`. This is a Julia type that combines numbers with physical units (like Newtons). Does iterating a `Range(1u"kg", 0.1u"kg", 10u"kg")` work as expected? Make the necessary adjustions so that it does.

## 4.4. Rational numbers

Implement the type `RationalNumber`, similar to Julia's `Rational`, but without being a parametric type (i.e. both fields are just `Int`). Using the following function
```julia
function _normalize(n::Integer, d::Integer)
    g = gcd(n, d)
    m = d < 0 ? -1 : 1
    (m * n ÷ g, m * d ÷ g)
end
```
define an **inner constructor**: a function `RationalNumber` inside the type-definition, which returns `new(a,b)`. (`new` stands for the type to be created). For example:
```julia
struct A
    a::Float64
    function A(a)
        x = cos(a)
        return new(x)
    end
end
```
Inner constructors are used to perform specific tests or transformations necessary before instantiating a type.

Then, implement `+, -, *, /` for `RationalNumber`.

Then, implemement "pretty printing" for this type, by extending `Base.show(io::IO, r::Rational)`, and using inside it `Base.print(io, r)`.